In [1]:
import pandas as pd

In [2]:
import pandas as pd

#엑셀 파일 불러오기
my_car_path = '/home/dong/dev_ws/eda/project_test/p_data/구별_승용차_현황.xlsx' 

my_car = pd.read_excel(my_car_path, engine='openpyxl')

#필요한 열만 선택
my_car = my_car[['시군구별', '계']].copy()

#'계' 열을 숫자로 변환 (문자 → 숫자, 오류는 NaN으로 처리)
my_car['계'] = pd.to_numeric(my_car['계'], errors='coerce')

#'합계' 또는 '소계'가 포함된 행 제거
my_car = my_car[~my_car['시군구별'].astype(str).str.contains('합계|소계', na=False)]

# NaN 값 제거
my_car = my_car.dropna()

#자치구별 등록 대수 합계 계산
df_my_car = my_car.groupby('시군구별', as_index=False)['계'].sum()

#컬럼명 정리
df_my_car.rename(columns={'시군구별': '구', '계': '승용차등록대수'}, inplace=True)

df_my_car

,구,승용차등록대수
0,강남구,254051
1,강동구,166203
2,강북구,73634
3,강서구,202214
4,관악구,116612
5,광진구,95829
6,구로구,145248
7,금천구,90653
8,노원구,92329
9,도봉구,152978


In [3]:
df_my_car = df_my_car.iloc[0:25,:]
df_my_car

,구,승용차등록대수
0,강남구,254051
1,강동구,166203
2,강북구,73634
3,강서구,202214
4,관악구,116612
5,광진구,95829
6,구로구,145248
7,금천구,90653
8,노원구,92329
9,도봉구,152978


In [4]:
df_my_car.to_csv("구별_승용차등록대수.csv", index=False, encoding='utf-8')

-----------------------------
# 구를 sort_value로 정렬하고, 구를 구_ID로 바꾼다

In [5]:
df_my_car

,구,승용차등록대수
0,강남구,254051
1,강동구,166203
2,강북구,73634
3,강서구,202214
4,관악구,116612
5,광진구,95829
6,구로구,145248
7,금천구,90653
8,노원구,92329
9,도봉구,152978


In [10]:
df_my_car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   구        25 non-null     object
 1   승용차등록대수  25 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 528.0+ bytes


In [6]:
import mysql.connector

gu_list = ['강남구', '강동구', '강북구', '강서구', '관악구', 
           '광진구', '구로구', '금천구', '노원구', '도봉구', 
           '동대문구', '동작구', '마포구', '서대문구', '서초구', 
           '성동구', '성북구', '송파구', '양천구', '영등포구', 
           '용산구', '은평구', '종로구', '중구', '중랑구']

conn = mysql.connector.connect(
host = "database-1.cb4o0sikaygu.ap-northeast-2.rds.amazonaws.com",
port = 3306,
user = "root",
password = "root762462",
database = "edadb"
)

In [11]:
df_db =df_my_car.copy()
for i in range(len(gu_list)):
    df_db.iloc[i,0] = i

df_db['구']=df_db['구'].astype(int)

-------------
# 테이블 수정

In [12]:
##### 바꿀 테이블로 이름 바꾸기.
sql = "insert into gu_car_count values (%s,%s)"
cursor = conn.cursor(buffered=True)

In [13]:
import numpy as np

def convert_types(row):
    new_row = []
    for item in row:
        if isinstance(item, (np.int64, np.integer)):
            new_row.append(int(item))
        elif isinstance(item, (np.float64, np.floating)):
            new_row.append(float(item))
        else:
            new_row.append(item)
    return tuple(new_row)

data = [convert_types(row) for row in df_db.values]

cursor.executemany(sql, data)
conn.commit()